In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re
import urllib.parse

In [2]:
input = pd.read_csv('hinatazaka_blog.csv', encoding="utf8")

In [3]:
removeUrls =[
    "https://cdn.hinatazaka46.com/files/14/hinata/img/logo_side.svg",
    "https://cdn.hinatazaka46.com/files/14/hinata/img/bg_ diamond.svg",
    "https://cdn.hinatazaka46.com/images/14/465/4b5f71e7f4d4f0d796dbeab7159ac.jpg",
    "https://cdn.hinatazaka46.com/images/14/ed9/236d2c08dbda29708de4b1f081bf3.jpg",
    "https://cdn.hinatazaka46.com/images/14/c57/5453a58d5c8963c907c1b2e9c6c88.jpg",
    "https://cdn.hinatazaka46.com/files/14/hinata/img/icon_logo_radius.svg",
    "https://cdn.hinatazaka46.com/files/14/hinata/img/jasrac.jpg"
]

In [ ]:
CURRENT = 'https://www.hinatazaka46.com'
# パターンでURL識別(最後のページ用)
pattern = "cd=member"

for i in range(len(input)):
    START = 'https://www.hinatazaka46.com/s/official/diary/member/list?ima=0000&page=0&cd=member&ct=' + str(input["ct"][i])
    soup = BeautifulSoup(requests.get(START).text, 'html.parser')
    
    # ブログページのリスト とりあえず最初のページを追加する
    blog_pages = [START]
    
    URL = START
    while True:
        if URL != START:
            soup = BeautifulSoup(requests.get(URL).text, 'html.parser')

        links = []
        for link in soup.findAll("a"):
            if "/s/official/diary/member/list?ima=0000" in link.get("href"):
                links.append(CURRENT + link.get("href"))

        all_loop = True
        # 現在のページの次ページを見つける
        for link in links:
            URLQueryString = urllib.parse.urlparse(URL).query
            linkQueryString = urllib.parse.urlparse(link).query
            URLQueryString_d = urllib.parse.parse_qs(URLQueryString)
            linkQueryString_d = urllib.parse.parse_qs(linkQueryString)

            if not 'page' in linkQueryString_d.keys():
                continue

             # page=xx の部分を比較してる +1のが見つかったらblog_pagesに追加してfor抜ける
            if int(URLQueryString_d['page'][0]) + 1 == int(linkQueryString_d['page'][0]):
                URL = link
                blog_pages.append(URL)
                all_loop = False
                break

        if all_loop:
            break
    
    targets = []
    for blog_page in blog_pages:
        soup = BeautifulSoup(requests.get(blog_page).text, 'html.parser')
        urls = []
        for link in soup.find_all("img"):
            if link.get("src"):
                src = link.get("src")
                urls.append(src)
        correctUrls = [url for url in urls if url not in removeUrls]
        targets.extend(correctUrls)
    
    # 重複削除
    targets = sorted(set(targets), key=targets.index)
    
    folderPath = "images/" + input["name"][i] + "/"
    os.makedirs(folderPath, exist_ok=True)
    
    # 画像保存
    for target in targets:
        filePath = folderPath + target.split("/")[-2] + "_" + target.split("/")[-1]
        if not os.path.exists(filePath):
            with open(filePath, "wb") as fp:
                res = requests.get(target)
                fp.write(res.content)